"""
Dependencies: numpy 1.17, pandas 0.25

userUserNetwork = W; Weighted adjacency matrix, probability user u respond to user v
treads = R
posts = P : user u, creation t, text x
documents = N
questionTendency = average; number of questions by total posts by user u in thread r for topic k
seeking (question) = S; QuestionTendency * log of 1+posts*length
disseminating (answer) = D; 1-Seeking
dictionary = X
topics = K
postTopics (theta) = [0,1]^N*K
topicWords = [0,1]^K*X
SIDR = phi; proportion of seeking by u on topic k by probability for user v responds to user u on topic k
DISR = psi; proportion of disseminating by u on topic k by probability for user u responds to user v on topic k
Benefit = B; utility obtained by user u for topic k; seeking*log of prob v to u on topic k
alpha = marginal benefit of teaching
smoothing = sigma
c_S, c_D = tightness parameters
step = lambda
t = threshold; error

Compute User-User Network
1-Smooth to ensure user responds to each post at most once
QuestionTendency = proportion of questions per topic per thread per weighted-average Q for u

Seeking and Dissemination
1:Extract forum topics: remove stopwords, urls, stem, lemmatize 
2:Infer if post is question or answer: first post and; other post or; has question mark or 5W1H or 1G
3:Compute S and D

Projected Gradient Descent
C_s = participation rate for seeking
C_d = participation rate for dissemination
alpha = learning step
"""

In [ ]:
thread = pd.read_json(r'/home/davidlemay/Documents/social_learning_network_analysis/Coursera MOOCs/courses/designingcities-001/thread250-0',orient='records',typ='series',convert_dates=False)

In [ ]:
thread = {}
with open ('/home/davidlemay/Documents/social_learning_network_analysis/Coursera MOOCs/courses/designingcities-001/thread25-0', 'r') as f:
    thread = json.loads(f.read())

In [ ]:
json.load(open('/home/davidlemay/Documents/social_learning_network_analysis/Coursera MOOCs/courses/designingcities-001/thread25-0','r'))

In [1]:
import pandas as pd
import numpy as np
#import proxmin as px
import json, os

In [248]:
#Import Coursera course discussion forum files, concatenate threads
directory = r'/home/davidlemay/Documents/social_learning_network_analysis/Coursera MOOCs/courses/designingcities-001/'
threads = []
for f in os.listdir(directory):
    thread = json.load(open(os.path.join(directory,f),'r'))
    posts = pd.DataFrame(thread['posts'])
    posts['parent_id'] = posts.sort_values(['thread_id','order'])['user_id'].shift(+1)
    posts['id'].apply(pd.to_numeric)
    comments = pd.DataFrame(thread['comments'])
    threads.append(posts)
    posts.set_index('id',inplace=True)
    if comments.size > 0:
        comments['post_id'].apply(pd.to_numeric)
        comments.set_index('id',inplace=True)
        comments['parent_id'] = pd.Series(comments['post_id'].apply(lambda x: posts.loc[x, 'user_id']))
        threads.append(comments)
    else:
        continue
df = pd.concat(threads)

In [ ]:
ids = []
for thread in threads:
    if 'id' in thread.columns:
        ids.append(thread['id'])
group = list(pd.concat(ids))
assert(len(group) - len(list(set(group))) == 0)

In [249]:
df.head()

,order,thread_id,user_id,post_time,edit_time,deleted,is_spam,original,stickied,approved,...,_viewer_can_vote,_viewer_can_edit,_user_title,_user_full_name,_user_profile_link,link,_reporter_link,parent_id,post_id,comment_text
id,,,,,,,,,,,,,,,,,,,,,
180,1.0,24.0,3294914.0,1.381146e+09,-1.0,0.0,0.0,1.0,0.0,0.0,...,True,False,Student,Ankita Biswas,https://class.coursera.org/designingcities-001...,https://class.coursera.org/designingcities-001...,https://class.coursera.org/designingcities-001...,NaN,NaN,NaN
795,2.0,24.0,5039265.0,1.381330e+09,-1.0,0.0,0.0,0.0,0.0,0.0,...,True,False,Student,Raj Shekhar Mishra,https://class.coursera.org/designingcities-001...,https://class.coursera.org/designingcities-001...,https://class.coursera.org/designingcities-001...,3294914.0,NaN,NaN
2219,1.0,220.0,5137894.0,1.382688e+09,-1.0,0.0,0.0,1.0,0.0,0.0,...,True,False,Student,Agnieszka Skóra,https://class.coursera.org/designingcities-001...,https://class.coursera.org/designingcities-001...,https://class.coursera.org/designingcities-001...,NaN,NaN,NaN
2271,1.0,230.0,4464254.0,1.382881e+09,-1.0,0.0,0.0,1.0,0.0,0.0,...,True,False,Student,Anonymous,https://class.coursera.org/designingcities-001...,https://class.coursera.org/designingcities-001...,https://class.coursera.org/designingcities-001...,NaN,NaN,NaN
1,1.0,1.0,3498045.0,1.380038e+09,-1.0,0.0,0.0,1.0,0.0,0.0,...,True,False,Staff,Meng Cao,https://class.coursera.org/designingcities-001...,https://class.coursera.org/designingcities-001...,https://class.coursera.org/designingcities-001...,NaN,NaN,NaN


In [245]:
df[df['comment_text'].isna() == False].sort_values(['thread_id','order'])

,order,thread_id,topics,user_id,post_time,edit_time,deleted,is_spam,original,stickied,...,_viewer_can_edit,_user_title,_user_full_name,_user_profile_link,link,_reporter_link,parent_id,post_id,comment_text,q_a
id,,,,,,,,,,,,,,,,,,,,,
80,61.0,1.0,0,4433273.0,1.381119e+09,-1.0,1.0,0.0,0.0,0.0,...,False,Student,Yu Sun,https://class.coursera.org/designingcities-001...,https://class.coursera.org/designingcities-001...,https://class.coursera.org/designingcities-001...,4137180.0,NaN,,None
80,61.0,1.0,0,4433273.0,1.381119e+09,-1.0,1.0,0.0,0.0,0.0,...,False,Student,Yu Sun,https://class.coursera.org/designingcities-001...,https://class.coursera.org/designingcities-001...,https://class.coursera.org/designingcities-001...,NaN,NaN,,None
105,73.0,1.0,0,5043551.0,1.381128e+09,-1.0,1.0,0.0,0.0,0.0,...,False,Student,edwinyuu,https://class.coursera.org/designingcities-001...,https://class.coursera.org/designingcities-001...,https://class.coursera.org/designingcities-001...,5014551.0,NaN,,None
1,NaN,1.0,9,3883580.0,1.381086e+09,NaN,0.0,0.0,NaN,NaN,...,False,Staff,Xin Tan,https://class.coursera.org/designingcities-001...,https://class.coursera.org/designingcities-001...,https://class.coursera.org/designingcities-001...,1693442.0,2.0,Welcome to Designing Cities! Thank you for bei...,NaN
18,NaN,1.0,2,4338315.0,1.381138e+09,NaN,0.0,0.0,NaN,NaN,...,False,Student,Anonymous,https://class.coursera.org/designingcities-001...,https://class.coursera.org/designingcities-001...,https://class.coursera.org/designingcities-001...,1693442.0,2.0,Hi Everybody.&nbsp;<br /><br />Hello. I am a c...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2240,NaN,485.0,0,2240869.0,1.388861e+09,NaN,0.0,0.0,NaN,NaN,...,False,Student,"Jeffrey Woo, MAED; EdD",https://class.coursera.org/designingcities-001...,https://class.coursera.org/designingcities-001...,https://class.coursera.org/designingcities-001...,0.0,3717.0,"sorry about that, anonymous",NaN
2242,NaN,485.0,0,0.0,1.389045e+09,NaN,0.0,0.0,NaN,NaN,...,False,NaN,NaN,NaN,https://class.coursera.org/designingcities-001...,https://class.coursera.org/designingcities-001...,820444.0,3722.0,when did you submitted your assignment 3?,NaN
2243,NaN,485.0,0,0.0,1.389045e+09,NaN,0.0,0.0,NaN,NaN,...,False,NaN,NaN,NaN,https://class.coursera.org/designingcities-001...,https://class.coursera.org/designingcities-001...,820444.0,3722.0,"Sergio, my advice is to create a new discussio...",NaN


In [238]:
df[df['comment_text'].isnull() == True]['comment_text'].size

8015

In [239]:
total_posts = 0
for x in threads:
    total_posts += len(x)
print(total_posts)

10033


In [244]:
df.loc[2320,'user_id']

1305506.0

In [241]:
df[df['post_text'].str.contains('textbook') == True]

,order,thread_id,topics,user_id,post_time,edit_time,deleted,is_spam,original,stickied,...,_viewer_can_edit,_user_title,_user_full_name,_user_profile_link,link,_reporter_link,parent_id,post_id,comment_text,q_a
id,,,,,,,,,,,,,,,,,,,,,
890,12.0,57.0,7,4786225.0,1.381359e+09,-1.0,0.0,0.0,0.0,0.0,...,False,Student,Sheng Jiang,https://class.coursera.org/designingcities-001...,https://class.coursera.org/designingcities-001...,https://class.coursera.org/designingcities-001...,4915487.0,NaN,NaN,question
1262,25.0,57.0,9,4786225.0,1.381604e+09,-1.0,0.0,0.0,0.0,0.0,...,False,Student,Sheng Jiang,https://class.coursera.org/designingcities-001...,https://class.coursera.org/designingcities-001...,https://class.coursera.org/designingcities-001...,4902399.0,NaN,NaN,question
434,5.0,28.0,0,1481139.0,1.381208e+09,-1.0,0.0,0.0,0.0,0.0,...,False,Student,Erin Partlan,https://class.coursera.org/designingcities-001...,https://class.coursera.org/designingcities-001...,https://class.coursera.org/designingcities-001...,4812761.0,NaN,NaN,question
2503,18.0,236.0,0,467106.0,1.383204e+09,-1.0,0.0,0.0,0.0,0.0,...,False,Student,Ann Priestley,https://class.coursera.org/designingcities-001...,https://class.coursera.org/designingcities-001...,https://class.coursera.org/designingcities-001...,0.0,NaN,NaN,question


In [193]:
df[df['post_id'].isnull() == True].index

Int64Index([ 180,  795, 2219, 2271,    1,    2,    4,    5,    6,    7,
            ...
            2798, 2799, 2800, 2801, 3560, 3563, 3572, 3580, 3607, 3609],
           dtype='int64', name='id', length=8121)

In [250]:
import re

def q_a(x):
    qWords = re.compile(r'[\w\W]*(who|what|where|when|why|how|\?)[\w\W]*')
    if re.search(qWords, x):
        return 'question'
    elif x != '':
        return 'answer'
    else:
        return None
        
df['q_a'] = df['post_text'].apply(lambda x: q_a(x) if type(x) == str else np.nan)

In [ ]:
# EdX transforms
#df['_id'] = df['_id'].transform(lambda x: x['$oid'])
#df['parent_id'] = df['parent_id'].transform(lambda x: x['$oid'] if type(x) == dict else np.nan)

In [ ]:
# EdX transform
#df.set_index('_id', inplace=True)
#df['parent_author_id'] = pd.Series(df['parent_id'].apply(lambda x: df.loc[x, 'author_id'] if type(x) == str else 0), dtype='Int64')
#df.reset_index(inplace=True)

In [196]:
df.to_csv('designingcities-forum.csv')

In [ ]:
#df['post_text'] = df['post_text'].transform(lambda x: x if type(str) else "")

In [246]:
# _type : either CommentThread (i.e., initiation) or Comment (i.e., reply)
InitiationsReplies = pd.crosstab(df['user_id'], df['q_a'])
#InitiationsReplies.head()

In [197]:
# https://github.com/susanli2016/NLP-with-Python/blob/master/LDA_news_headlines.ipynb
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import nltk
#nltk.download('wordnet')

In [198]:
stemmer = SnowballStemmer('english')
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [199]:
len(df['post_text'])

10033

In [315]:
#print(WordNetLemmatizer().lemmatize(gensim.utils.simple_preprocess(df['post_text'][1])[7], pos='v'))

In [ ]:
#gensim.utils.simple_preprocess(df['post_text'][1])

In [204]:
processed_docs = df['post_text'].map(lambda x: preprocess(x) if type(x) == str else [])

In [205]:
processed_docs[:10]

id
180     [hello, livabl, citi, happi, index, relat, ter...
795     [measur, liveabl, happi, meaning, capita, heal...
2219    [span, style, color, font, size, line, height,...
2271    [current, global, urbanis, largest, movement, ...
1       [welcom, design, citi, general, forum, warm, i...
2       [like, thank, involv, make, onlin, cours, espe...
4       [herbert, munich, germani, aerospac, engin, pu...
5       [hello, thank, cours, christiana, studi, chemi...
6       [hello, everybodi, pieter, netherland, citi, p...
7                            [greet, hello, philadelphia]
Name: post_text, dtype: object

In [206]:
dictionary = gensim.corpora.Dictionary(processed_docs)

In [ ]:
#dictionary.filter_extremes(no_below=5, no_above=0.5, keep_n=100000)

In [207]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

In [208]:
len(processed_docs)

10033

In [209]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2)

In [210]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.035*"nbsp" + 0.024*"citi" + 0.014*"http" + 0.010*"design" + 0.008*"href" + 0.008*"work" + 0.008*"peopl" + 0.007*"urban" + 0.007*"https" + 0.006*"target"
Topic: 1 
Words: 0.025*"cours" + 0.024*"assign" + 0.011*"nbsp" + 0.010*"think" + 0.009*"grade" + 0.008*"time" + 0.008*"good" + 0.007*"citi" + 0.007*"like" + 0.007*"certif"
Topic: 2 
Words: 0.017*"nbsp" + 0.016*"citi" + 0.012*"cours" + 0.011*"plan" + 0.008*"thread" + 0.008*"coursera" + 0.007*"peopl" + 0.007*"thank" + 0.007*"urban" + 0.007*"work"
Topic: 3 
Words: 0.021*"http" + 0.019*"citi" + 0.013*"plan" + 0.013*"nbsp" + 0.011*"href" + 0.011*"urban" + 0.009*"target" + 0.008*"link" + 0.008*"design" + 0.007*"think"
Topic: 4 
Words: 0.045*"citi" + 0.010*"http" + 0.010*"urban" + 0.009*"develop" + 0.008*"design" + 0.008*"nbsp" + 0.007*"plan" + 0.006*"area" + 0.005*"build" + 0.005*"cours"
Topic: 5 
Words: 0.024*"http" + 0.014*"assign" + 0.012*"nbsp" + 0.009*"href" + 0.009*"citi" + 0.008*"target" + 0.007*"grade" + 0.007*"lin

In [211]:
topicScores = [lda_model.get_document_topics(x) for x in bow_corpus]

In [264]:
topics = pd.DataFrame([max(x, key=lambda y: y[1]) for x in topicScores])

In [265]:
df.insert(2, 'topics', topics[0])
#df.drop('topics', 1)
#topics.shape

In [266]:
len(df.post_text)

10033

In [215]:
df.columns

Index(['order', 'thread_id', 'topics', 'user_id', 'post_time', 'edit_time',
       'deleted', 'is_spam', 'original', 'stickied', 'approved', 'anonymous',
       'votes', 'post_text', 'user_agent', 'text_type', '_viewer_can_vote',
       '_viewer_can_edit', '_user_title', '_user_full_name',
       '_user_profile_link', 'link', '_reporter_link', 'parent_id', 'post_id',
       'comment_text', 'q_a'],
      dtype='object')

In [257]:
"""Create the adjacency matrix using _id for index and column labels"""
df1 = df.groupby(['post_id','user_id','parent_id'], sort=False, as_index=False)['user_id'].first()
userNetwork = pd.crosstab(df1.user_id,df1.parent_id)
idx = userNetwork.columns.union(userNetwork.index)
userNetwork = userNetwork.reindex(index=idx, columns=idx, fill_value=0)

In [283]:
df1[df1['post_id'] == 250]

,post_id,parent_id,user_id
1027,250.0,4206285.0,4926662.0
1028,250.0,4206285.0,0.0
1030,250.0,4206285.0,4684101.0
1032,250.0,4206285.0,5044037.0


In [258]:
userNetwork = pd.DataFrame(userNetwork.sort_index(axis=0).sort_index(axis=1))

In [220]:
#userNetwork.drop(0, axis=0, inplace=True)
#userNetwork.drop(0, axis=1, inplace=True)
#userNetwork.shape

In [259]:
#totalResponses = userNetwork.sum(axis=1)
totalPosts = userNetwork.sum(axis=0).transpose()
fracPosts = userNetwork/totalPosts

In [260]:
#w_UserNetwork = (fracPosts + (fracPosts.sum()/totalPosts.sum()))/(totalPosts + max(totalPosts))
w_UserNetwork = fracPosts

In [261]:
userNetwork.shape

(789, 789)

In [262]:
w_UserNetwork

,0.0,4004.0,4249.0,9991.0,37799.0,45460.0,76110.0,100472.0,186673.0,211857.0,...,5275645.0,5288942.0,5339521.0,5346233.0,5359214.0,5417040.0,5482892.0,5509229.0,5517162.0,5611124.0
0.0,0.246377,NaN,NaN,0.0,NaN,NaN,0.0,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,0.5,0.0,0.000000,NaN,0.0,NaN
4004.0,0.000000,NaN,NaN,0.0,NaN,NaN,0.0,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,0.0,0.0,0.000000,NaN,0.0,NaN
4249.0,0.000000,NaN,NaN,0.0,NaN,NaN,0.0,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,0.0,0.0,0.000000,NaN,0.0,NaN
9991.0,0.000000,NaN,NaN,0.0,NaN,NaN,0.0,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,0.0,0.0,0.000000,NaN,0.0,NaN
37799.0,0.000000,NaN,NaN,0.0,NaN,NaN,0.0,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,0.0,0.0,0.000000,NaN,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5417040.0,0.000000,NaN,NaN,0.0,NaN,NaN,0.0,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,0.0,0.5,0.000000,NaN,0.0,NaN
5482892.0,0.000000,NaN,NaN,0.0,NaN,NaN,0.0,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,0.0,0.0,0.333333,NaN,0.0,NaN
5509229.0,0.000000,NaN,NaN,0.0,NaN,NaN,0.0,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,0.0,0.0,0.000000,NaN,0.0,NaN
5517162.0,0.000000,NaN,NaN,0.0,NaN,NaN,0.0,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,0.0,0.0,0.000000,NaN,0.0,NaN


In [267]:
userTopics = pd.crosstab(df['user_id'], [df['q_a'], df['topics']])

In [268]:
userTopics.shape

(1141, 20)

In [269]:
postTopics = userTopics.sum(0)/userTopics.sum(0).sum()

In [270]:
postTopics

q_a       topics
answer    0         0.396525
          1         0.020478
          2         0.010549
          3         0.015203
          4         0.008377
          5         0.004654
          6         0.009308
          7         0.022960
          8         0.004034
          9         0.013962
question  0         0.380081
          1         0.018926
          2         0.015824
          3         0.008998
          4         0.008067
          5         0.005585
          6         0.013342
          7         0.024511
          8         0.004654
          9         0.013962
dtype: float64

In [271]:
postingTendency = userTopics['question']*postTopics.sum()+userTopics['answer']*postTopics.sum()
questionTendency = (userTopics['question']*postTopics.sum())/postingTendency.sum()

In [272]:
questionTendency.shape

(1141, 10)

In [273]:
#userTopics.columns = userTopics.columns.droplevel(0)
disseminating_full = (1-questionTendency * np.log(1+postingTendency))
seeking_full = (questionTendency * np.log(1+postingTendency))

In [274]:
idx = list(set(userNetwork.index.values).intersection(disseminating_full.index.values))

In [275]:
disseminating = disseminating_full.filter(items=idx,axis=0)
seeking = seeking_full.filter(items=idx,axis=0)
f_UserNetwork = w_UserNetwork.filter(items=idx,axis=0).filter(items=idx,axis=1)

In [276]:
f_UserNetwork.shape

(645, 645)

In [313]:
threshold = .01
alpha = .4
beta = 0.8
c_S = 1.25
c_D = 0.75
step = 0.1
rho = 1
N = f_UserNetwork.shape[0]
z1 = lambda1 = np.zeros(seeking.T.shape).astype('float64')
z2 = lambda2 = np.zeros(disseminating.shape).astype('float64')
W = W_hat = W_obs = f_UserNetwork.fillna(0).astype('float64').to_numpy()
W_opt = W_prime = np.zeros(W.shape)
D = disseminating.fillna(0).astype('float64').to_numpy()
S = seeking.fillna(0).astype('float64').to_numpy()

def SIDR(s,d,w):
    phi = np.zeros(s.shape)
    for u in range(0,s.shape[0]):
        for k in range(0,s.shape[1]):
            a = s[u,k]
            b = w.T[:,u].sum()*d[:,k].sum()
            phi[u,k] = np.divide(a,b, out=np.zeros_like(a), where=b!=0)
    return phi

def DISR(s,d,w):
    psi = np.zeros(d.shape)
    for u in range(d.shape[0]):
        for k in range(d.shape[1]):
            a = d[u,k]
            b = w[u,:].sum()*s[:,k].sum()
            psi[u,k] = np.divide(a,b, out=np.zeros_like(a), where=b!=0)
    return psi

    
#SIDR.replace(to_replace=0,value=1)
#DISR.replace(to_replace=0,value=1)
phi = c_S*SIDR(S,D,W)
psi = c_D*DISR(S,D,W)
P = np.divide(S,phi, out=np.zeros_like(S), where=phi!=0).T
Q = np.divide(D,psi, out=np.zeros_like(D), where=psi!=0)

In [314]:
np.seterr(all="raise")

def benefit(X):
    b = np.zeros(D.shape)
    for u in range(D.shape[0]):
        for k in range(D.shape[1]):
            b[u,k] = (S[u,k]*np.log(1+X[u].sum()*D[k].sum()) + alpha*D[u,k]*np.log(1+X[u].sum()*S[k].sum()))
    return b

def proj(X):
    """Projection step"""
    #return np.subtract(X,np.diag(np.diag(X)))/X.shape[0]/np.linalg.norm(X)
    return np.clip(np.subtract(X,np.diag(np.diag(X))),0,1)
    
def grad(X):
    """Proximal gradient step"""
    return (np.add((rho*np.dot(D,(np.add(np.subtract(np.dot(D.T,X),P),np.subtract(z1,lambda1))))),
            (rho*np.dot((np.subtract(np.subtract(np.dot(X,S),Q),np.add(z2,lambda2))),S.T))))

g = benefit(W).sum().sum()
g_hat = 1
i = 0

while (g - g_hat)/np.abs(g_hat) >= threshold:
    i += 1
    for u in range(W.shape[0]):
        for v in range(W.shape[1]):
            W_prime[u,v] = (
                (D[u].sum()*S[v].sum()/(1+(W[v].sum()*D.sum()).sum())
                +alpha*D[u].sum()*S[v].sum()/(1+(W[u].sum()*S.sum()).sum()))
                /N
            )
    W_hat = (W + step * W_prime)
    p = proj(np.subtract(W,step*(np.subtract(W,np.add(W_hat,grad(W))))))
    W = p
    #W = proj(grad(W_hat))
    g_hat = g
    g = benefit(W).sum().sum()
    z1 = np.maximum(np.add(np.add((-1*np.dot(D.T,W)),P),lambda1),0)
    z2 = np.maximum(np.add(np.subtract(np.dot(W,S),Q),lambda2),0)
    lambda1 -= np.subtract(np.add(np.dot(D.T,W),P),z1)
    lambda2 += np.subtract(np.subtract(np.dot(W,S),Q),z2)
    step *= beta

print("Iterations: "+str(i))
print("Observed learning benefit: "+str(benefit(W_obs).sum().sum()/N))
print("Optimized learning benefit: "+str(g_hat/N))

Iterations: 3
Observed learning benefit: 0.17861350484497948
Optimized learning benefit: 3.8120391605620876


In [302]:
from numpy import linalg as la
import networkx as nx

In [ ]:
nx.local_efficiency(nx.from_numpy_array(W))

In [315]:
nx.global_efficiency(nx.from_numpy_array(W))

0.9883215369059656

In [ ]:
np.max(la.eig(W_obs)[0]).real

In [ ]:
np.max(la.eig(W)[0]).real

In [316]:
np.savetxt('designingcities-001-optimized.csv',W,delimiter=',')